As a kind of smoke test, I've been provided with some test cases to verify the functionality of different components of the neural network as they're being created. We'll instantiate the components in the required configuration for the tests, and then observe whether they perform as expected.

In [81]:
import unittest
import itertools
from typing import Callable
from inspect import signature

import numpy as np
import pandas as pd

import ml

<h2>Perceptron</h2>

First off, we require tests the implemented perceptron unit. The provided scenarios to verify compliance with the expected behaviour of perceptron are several logical gates, consisting of either one or a network of perceptrons. The following scenarios are expected:

<ol>
<li>An INVERT gate</li>
<li>An AND gate</li>
<li>An OR gate</li>
<li>A NOR gate</li>
<li>A non-specified "more complex" decision system with at least 3 inputs.</li>
</ol>

We'll first establish a framework that allows us to easily construct truth tables for boolean operations. Then we'll compare these with the results from our perceptrons to verify their validity.


In [82]:
def binary_input_space(length: int) -> pd.DataFrame:
    return pd.DataFrame(list(itertools.product([0, 1], repeat=length))).rename(columns=lambda n: f"Input {n}")

In [83]:
def apply_operation(input_space: pd.DataFrame,
                    operation: Callable) -> pd.Series:
    return input_space.apply(lambda row: operation(*row), axis=1)

def apply_perceptron(input_space: pd.DataFrame,
                     perceptron: ml.Perceptron):
    return input_space.apply(lambda row: perceptron.activate(row), axis=1, raw=True)

In [84]:
def verify_perceptron_operation_inputs(operation: Callable,
                                       perceptron: ml.Perceptron) -> None:
    if len(signature(operation).parameters) != perceptron.weights.size:
        raise ValueError(f"""Operation amount of arguments do not match perceptron
expected amount of inputs.""")

In [85]:
def perceptron_truth_table(operation: Callable,
                           perceptron: ml.Perceptron) -> pd.DataFrame:
    verify_perceptron_operation_inputs(operation, perceptron)

    input_space = binary_input_space(perceptron.weights.size)

    true_result = apply_operation(input_space, operation)
    perceptron_result = apply_perceptron(input_space, perceptron)

    input_space["True Result"] = true_result
    input_space["Perceptron Result"] = perceptron_result

    return input_space


<h4>1. INVERT gate</h4>

The invert gate is simple, it takes one input, and has one output. It should always return the invert (0 -> 1 ∧ 1 -> 0). We can use the framework we established earlier to verify the correctness.

In [86]:
invert_weights = np.array([-1])
invert_bias = 0.5

invert_gate = ml.Perceptron(invert_weights, invert_bias)

In [87]:
invert_table = perceptron_truth_table(lambda a: int(not a),
                                      invert_gate)

invert_table

,Input 0,True Result,Perceptron Result
0,0,1,1
1,1,0,0


In [88]:
(invert_table["True Result"] == invert_table["Perceptron Result"]).all()

True

<h4>2. AND gate</h4>

In [89]:
and_weights = np.array([0.5, 0.5])
and_bias = -1

and_gate = ml.Perceptron(and_weights, and_bias)

In [90]:
and_table = perceptron_truth_table(lambda a, b: int(a and b),
                                   and_gate)

and_table

,Input 0,Input 1,True Result,Perceptron Result
0,0,0,0,0
1,0,1,0,0
2,1,0,0,0
3,1,1,1,1


In [91]:
(and_table["True Result"] == and_table["Perceptron Result"]).all()

True

<h4>3. OR gate</h4>

In [92]:
or_weights = np.array([1, 1])
or_bias = -1

or_gate = ml.Perceptron(or_weights, or_bias)

In [93]:
or_table = perceptron_truth_table(lambda a, b: int(a or b),
                                   or_gate)

or_table

,Input 0,Input 1,True Result,Perceptron Result
0,0,0,0,0
1,0,1,1,1
2,1,0,1,1
3,1,1,1,1


In [94]:
(or_table["True Result"] == or_table["Perceptron Result"]).all()

True

<h4>4. NOR gate</h4>

In [95]:
nor_weights = np.array([-1, -1, -1])
nor_bias = 0

nor_gate = ml.Perceptron(nor_weights, nor_bias)

In [96]:
nor_table = perceptron_truth_table(lambda a, b, c: int((not a) and (not b) and (not c)),
                                   nor_gate)

nor_table

,Input 0,Input 1,Input 2,True Result,Perceptron Result
0,0,0,0,1,1
1,0,0,1,0,0
2,0,1,0,0,0
3,0,1,1,0,0
4,1,0,0,0,0
5,1,0,1,0,0
6,1,1,0,0,0
7,1,1,1,0,0


In [97]:
(nor_table["True Result"] == nor_table["Perceptron Result"]).all()

True

<h4>5. A non-specified "more complex" decision system with at least 3 inputs.</h4>

For this, I'll be implementing something I'll call an <i>EVEN gate</i>